In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.metrics as metrics

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, AveragePooling1D, Conv1D, DenseFeatures, Input, Flatten, GlobalAveragePooling1D


# Loading Dataset

Our dataset contains four directories. Each directory referst to one class of motion to be classified. Each directory contains 395 recordings. 

---
## Recordings
Each line of the recording is the average value of the gyro sensor over 10 ms. The whole recording time is 2000ms. Therefore each recording has 200 lines.
Each line of the recording contais six different values. They are the acceleration values on all three spatial axes as well as the gyroscope information on all three spatial axes. 

> The accelerometer values are deliberately not cleaned from earths acceleration.
---
## Reading the data

For the reading of the data we iterate over all directories in the dataset directory, and the over each file in the gesture directory. We then save the data into a python list. We also save the desired classification outcome, at the same time, into another list. That way you will find the classification of the dataset at the same index of the label list as the data in the data list.


In [2]:
dataset_dir = "gestures"

gestures = ["Computer", "Elephant", "Sorry", "No-Sign"]

dataset_values = []
dataset_labels = []

for gesture in gestures:
    for filename in os.listdir(f"{dataset_dir}/{gesture}"):
        with open(os.path.join(f"{dataset_dir}/{gesture}", filename), 'r') as file:
            file_data = []

            text = file.read()
            lines = text.splitlines()

            for line in lines:
                int_values = []
                for value in line.split():
                    int_values.append(int(value))

                file_data.append(int_values)

            dataset_values.append(file_data)

            if gesture == "Computer":
                dataset_labels.append([1, 0, 0, 0])
            elif gesture == "Elephant":
                dataset_labels.append([0, 1, 0, 0])
            elif gesture == "Sorry":
                dataset_labels.append([0, 0, 1, 0])
            elif gesture == "No-Sign":
                dataset_labels.append([0, 0, 0, 1])
            else:
                print("You did not change the labeling of the dataset you blithering idiot")

dataset_values = np.array(dataset_values)
dataset_labels = np.array(dataset_labels)

print(dataset_values.shape)
print(dataset_labels.shape)

(1584, 200, 6)
(1584, 4)


## Creating the dataset

We convert the two lists mentioned above into a tf.Dataset. 

---

First we define SHUFFLE_BUFFER_SIZE and VALIDATION_SIZE. We want to shuffle the complete Dataset, therefore the shuffle buffer has the same size as the dataset itself. We want to have Validationset that is the size of about 30% of the whole Dataset.

We create our dataset and shuffle it.

For the validation set we take the defined amount from the front of the shuffled dataset. The rest is our training data.

In [3]:
DATASET_SIZE = len(dataset_values)

SHUFFLE_BUFFER_SIZE = DATASET_SIZE
VALIDATION_SIZE = int(DATASET_SIZE * 0.3)

BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((dataset_values, dataset_labels))
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)

validation_dataset = dataset.take(VALIDATION_SIZE).batch(BATCH_SIZE)  
train_dataset = dataset.skip(VALIDATION_SIZE).batch(BATCH_SIZE)

# for values, labels in train_dataset.take(1):
    # print(labels.numpy(), values.numpy())

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-05-20 10:26:24.846165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-20 10:26:24.846290: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## The Model

---

In [65]:
%%time
model = Sequential([
    Conv1D(filters = 200, strides = 3, kernel_size=70, activation='relu', input_shape=(200, 6)),
    GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(20, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["CategoricalAccuracy"])
model.build(input_shape=(1584, 200, 6))
model.summary()
model.fit(train_dataset, validation_data=validation_dataset, epochs=20)

model.save("./model", save_format='tf')
model.save("./model.h5", save_format='h5')


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_20 (Conv1D)          (None, 44, 200)           84200     
                                                                 
 global_average_pooling1d_20  (None, 200)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_60 (Dense)            (None, 100)               20100     
                                                                 
 dense_61 (Dense)            (None, 20)                2020      
                                                                 
 dense_62 (Dense)            (None, 4)                 84        
                                                                 
Total params: 106,404
Trainable params: 106,404
Non-trainable params: 0
_______________________________________________

2022-05-20 11:45:28.485894: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 1s 20ms/step - loss: 221.1909 - categorical_accuracy: 0.7592 - val_loss: 60.6996 - val_categorical_accuracy: 0.9032
Epoch 2/20
 6/18 [=========>....................] - ETA: 0s - loss: 53.8412 - categorical_accuracy: 0.8568

2022-05-20 11:45:28.895984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 0s 14ms/step - loss: 52.5972 - categorical_accuracy: 0.8846 - val_loss: 32.0938 - val_categorical_accuracy: 0.9242
Epoch 3/20
18/18 [==============================] - 0s 13ms/step - loss: 29.7313 - categorical_accuracy: 0.9333 - val_loss: 34.7673 - val_categorical_accuracy: 0.9368
Epoch 4/20
18/18 [==============================] - 0s 13ms/step - loss: 15.7303 - categorical_accuracy: 0.9531 - val_loss: 3.2768 - val_categorical_accuracy: 0.9789
Epoch 5/20
18/18 [==============================] - 0s 15ms/step - loss: 13.5930 - categorical_accuracy: 0.9603 - val_loss: 9.4216 - val_categorical_accuracy: 0.9663
Epoch 6/20
18/18 [==============================] - 0s 13ms/step - loss: 8.6021 - categorical_accuracy: 0.9684 - val_loss: 22.1594 - val_categorical_accuracy: 0.9516
Epoch 7/20
18/18 [==============================] - 0s 17ms/step - loss: 18.4910 - categorical_accuracy: 0.9495 - val_loss: 13.8623 - val_categorical_accuracy: 0.9642
Epoch 8/20
1

INFO:tensorflow:Assets written to: ./model/assets


CPU times: user 4.75 s, sys: 2.5 s, total: 7.26 s
Wall time: 5.54 s


In [47]:
modeldir = 'models/0.9900/model'
converter = tf.lite.TFLiteConverter.from_saved_model(modeldir)
tflite_model = converter.convert()

confusion = tf.confusion_matrix(validation_dataset)

open("model.tflite", "wb").write(tflite_model)

2022-05-20 11:21:32.227073: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-05-20 11:21:32.227086: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-05-20 11:21:32.227157: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: models/0.9900/model
2022-05-20 11:21:32.227901: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-05-20 11:21:32.227906: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: models/0.9900/model
2022-05-20 11:21:32.229951: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-05-20 11:21:32.255220: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: models/0.9900/model
2022-05-20 11:21:32.262338: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Too

324836

## Evaluation
---

In [156]:
import matplotlib.pyplot as plt


evaluation_results = model.evaluate(validation_dataset)
print("Loss: {}".format(evaluation_results[0]))
print("Accuracy: {}".format(evaluation_results[1]))

## Printing Confusion Matrix
labels = []
values = []
for element in validation_dataset.as_numpy_iterator():
  value, label = element
  values.append(value)
  labels.append(label)

predictions = []


def get_highest_value(list):
  highest_index = 0
  for i, val in enumerate(list):
    if val > list[highest_index]:
      highest_index = i

  return highest_index


clear_labels = []
clear_predicitons = []

for elem in values:
  predictions.append(model.predict(elem))

for j in range(len(labels)):
  for i in range(len(labels[j])):
    clear_labels.append(get_highest_value(labels[j][i]))
    clear_predicitons.append( get_highest_value(predictions[j][i]))


# for i, val in enumerate(clear_labels):
#   print(f"label: {clear_labels[i]} pred: {clear_predicitons[i]} eval: {clear_labels[i] == clear_predicitons[i]}")


labels = np.array(labels)  
predictions = np.array(predictions)  


print("\n Confusion Matrix of Validation Set:")
print(tf.math.confusion_matrix(clear_labels, clear_predicitons))

8/8 [==============================] - 0s 8ms/step - loss: 1.5143 - categorical_accuracy: 0.9768
Loss: 1.5142583847045898
Accuracy: 0.9768421649932861

 Confusion Matrix of Validation Set:
tf.Tensor(
[[120   1   0   1]
 [  0 126   0   0]
 [  0   0 115   0]
 [  2   0   0 110]], shape=(4, 4), dtype=int32)


/var/folders/lv/rc11t0614pd6l8fc81zjgd3h0000gn/T/ipykernel_6206/2063199296.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array(labels)
/var/folders/lv/rc11t0614pd6l8fc81zjgd3h0000gn/T/ipykernel_6206/2063199296.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(predictions)
